<a href="https://colab.research.google.com/github/AlexanderBelfort/MDPRKT/blob/main/SVMNB_GALL_VS_OTHERS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import pickle

# Set Random seed
np.random.seed(500)

# Add the Data using pandas

Corpus = pd.read_csv(r"corpus_dual_2.csv", encoding='latin-1')
#header=None, error_bad_lines=False)
'''col_name = Corpus.columns[0]
Corpus=Corpus.rename(columns = {col_name:'text'})

col_name2 = Corpus.columns[1]
Corpus=Corpus.rename(columns = {col_name2:'label'})
'''
Corpus.head()

,text,label
0,Cholesterol gallstones result from oversaturat...,__GALLSTONES__
1,Gallstones are highly prevalent in most develo...,__GALLSTONES__
2,Lung cancer does not usually cause noticeable ...,__OTHER_DISEASES__
3,Multiple studies on the subjet of gallstones h...,__GALLSTONES__
4,Research indicates that gallstones have been h...,__GALLSTONES__


In [19]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# Step - 1: Data Pre-processing - This will help in getting better results through the classification algorithms

# Step - 1a : Remove blank rows if any.
Corpus['text'].dropna(inplace=True)

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


def text_preprocessing(text):
    # Step - 1b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
    text = text.lower()

    # Step - 1c : Tokenization : In this each entry in the corpus will be broken into set of words
    text_words_list = word_tokenize(text)

    # Step - 1d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(text_words_list):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
            Final_words.append(word_Final)
        # The final processed set of words for each iteration will be stored in 'text_final'
    return str(Final_words)


Corpus['text_final'] = Corpus['text'].map(text_preprocessing)

# Step - 2: Split the model into Train and Test Data set
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'], Corpus['label'],
                                                                    test_size=0.3)

# Step - 3: Label encode the target variable  - This is done to transform Categorical data of string type in the data set into numerical values
Encoder = LabelEncoder()
Encoder.fit(Train_Y)
Train_Y = Encoder.transform(Train_Y)
Test_Y = Encoder.transform(Test_Y)

# Step - 4: Vectorize the words by using TF-IDF Vectorizer - This is done to find how important a word in document is in comaprison to the corpus
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

# Step - 5: Now we can run different algorithms to classify out data check for accuracy

# Classifier - Algorithm - Naive Bayes
# fit the training dataset on the classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf, Train_Y)

# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Test_Y) * 100)

# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf, Train_Y)

# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y) * 100)


# Saving Encdoer, TFIDF Vectorizer and the trained model for future infrerencing/prediction

# saving encoder to disk
filename = 'labelencoder_fitted.pkl'
pickle.dump(Encoder, open(filename, 'wb'))

# saving TFIDF Vectorizer to disk
filename = 'Tfidf_vect_fitted.pkl'
pickle.dump(Tfidf_vect, open(filename, 'wb'))

# saving the both models to disk
filename = 'svm_trained_model.sav'
pickle.dump(SVM, open(filename, 'wb'))

filename = 'nb_trained_model.sav'
pickle.dump(Naive, open(filename, 'wb'))

print("Files saved to disk! Proceed to inference.py")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Naive Bayes Accuracy Score ->  75.0
SVM Accuracy Score ->  83.33333333333334
Files saved to disk! Proceed to inference.py


In [26]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
import pickle
import ast

# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda: wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV


def text_preprocessing(text):
    # Step - 1b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
    text = text.lower()

    # Step - 1c : Tokenization : In this each entry in the corpus will be broken into set of words
    text_words_list = word_tokenize(text)

    # Step - 1d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(text_words_list):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
            Final_words.append(word_Final)
        # The final processed set of words for each iteration will be stored in 'text_final'
    return str(Final_words)


# Loading Label encoder
labelencode = pickle.load(open('labelencoder_fitted.pkl', 'rb'))

# Loading TF-IDF Vectorizer
Tfidf_vect = pickle.load(open('Tfidf_vect_fitted.pkl', 'rb'))

# Loading models
SVM = pickle.load(open('svm_trained_model.sav', 'rb'))
Naive = pickle.load(open('nb_trained_model.sav', 'rb'))


# Inference
sample_text = "Do I need to have my gallbladder removed?"
sample_text_processed = text_preprocessing(sample_text)
sample_text_processed_vectorized = Tfidf_vect.transform([sample_text_processed])

prediction_SVM = SVM.predict(sample_text_processed_vectorized)
prediction_Naive = Naive.predict(sample_text_processed_vectorized)

print("Prediction from SVM Model:", labelencode.inverse_transform(prediction_SVM)[0])
print("Prediction from NB Model:", labelencode.inverse_transform(prediction_Naive)[0])

Prediction from SVM Model: __GALLSTONES__
Prediction from NB Model: __GALLSTONES__


In [27]:
# Inference
sample_text = "How do you treat cancer?"
sample_text_processed = text_preprocessing(sample_text)
sample_text_processed_vectorized = Tfidf_vect.transform([sample_text_processed])

prediction_SVM = SVM.predict(sample_text_processed_vectorized)
prediction_Naive = Naive.predict(sample_text_processed_vectorized)

print("Prediction from SVM Model:", labelencode.inverse_transform(prediction_SVM)[0])
print("Prediction from NB Model:", labelencode.inverse_transform(prediction_Naive)[0])

Prediction from SVM Model: __OTHER_DISEASES__
Prediction from NB Model: __OTHER_DISEASES__


In [28]:
# Inference
sample_text = "I have bile problems."
sample_text_processed = text_preprocessing(sample_text)
sample_text_processed_vectorized = Tfidf_vect.transform([sample_text_processed])

prediction_SVM = SVM.predict(sample_text_processed_vectorized)
prediction_Naive = Naive.predict(sample_text_processed_vectorized)

print("Prediction from SVM Model:", labelencode.inverse_transform(prediction_SVM)[0])
print("Prediction from NB Model:", labelencode.inverse_transform(prediction_Naive)[0])

Prediction from SVM Model: __GALLSTONES__
Prediction from NB Model: __GALLSTONES__
